# 情感分析

## 在 SageMaker 中使用 XGBoost

_机器学习工程师纳米学位课程 | 开发_

---

在这个 Amazon SageMaker 服务使用示例中，我们将构建一个预测影评情感的随机树模型。你可能在上节课已经见过这个示例的一个版本，不过当时是用 sklearn 软件包实现的，现在我们将使用 Amazon 提供的 XGBoost 软件包。

## 说明

我们已经提供了一些模板代码，但是你需要实现其他功能，才能成功地完成此 notebook。除了要求的部分之外，不需要修改所包含的代码。标题以“**TODO**”开头的部分表示你需要完成或实现其中的某些部分。我们将在每个部分提供说明，并在代码块中用 `# TODO: ...` 注释标记出具体的实现要求。请务必仔细阅读说明。

除了实现代码之外，你还需要回答一些问题，这些问题与任务和你的实现代码有关。每个部分需要回答的问题都在标题中以“**问题：**”开头。请仔细阅读每个问题，并编辑下面以“**答案：**”开头的标记单元格，然后输入答案。

> 注意：可以通过 **Shift+Enter** 键盘快捷键执行代码和标记单元格。此外，通常还可通过点击单元格（标记单元格需要双击）编辑单元格，或者在选中后按下 **Enter** 键编辑单元格。

## 第 1 步：下载数据

我们要使用的数据集很受自然语言处理领域的研究者欢迎，通常称为 [IMDB 数据集](http://ai.stanford.edu/~amaas/data/sentiment/)。其中包含网站 [imdb.com](http://www.imdb.com/) 上的影评，每条影评都标有“**pos**itive”，表示评论者喜欢影片，否则标有“**neg**ative”。

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

我们先通过 Jupyter Notebook 功能下载和提取该数据集。

In [ ]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

## 第 2 步：准备和处理数据

我们下载的文件拆分成了各种文件，每个都包含一条影评。我们需要将这些文件合并成两个文件，一个用于训练，一个用于测试。

In [ ]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

In [ ]:
train_X[100]

## 第 3 步：处理数据

合并并准备好训练和测试数据集后，我们需要将原始数据处理成机器学习算法能够使用的格式。首先，删除所有的 HTML 格式标记，并执行一些标准自然语言处理步骤，使数据变得类同。

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
import re
from bs4 import BeautifulSoup

def review_to_words(review):
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [ ]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [ ]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

### 提取词袋特征

对于我们要实现的模型，我们并不直接使用影评，而是将每条影评转换成词袋特征表示法。注意，我们只能访问训练集，所以转换器只能使用训练集创建表示结果。

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size,
                preprocessor=lambda x: x, tokenizer=lambda x: x)  # already preprocessed
        features_train = vectorizer.fit_transform(words_train).toarray()

        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary

In [ ]:
# Extract Bag of Words features for both training and test datasets
train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)

## 第 4 步：使用 XGBoost 进行分类

创建了训练（和测试）数据的特征表示结果后，我们将开始设置和使用 SageMaker 提供的 XGBoost 分类器。

### 写入数据集

我们将使用的 XGBoost 分类器要求我们将数据集写入文件中并将文件存储到 Amazon S3 上。我们首先将训练数据集拆分成两部分，分别是训练集和验证集。然后，将这些数据集写入文件中，并将文件上传到 S3。此外，我们将测试集输入写入文件中并将文件上传到 S3。这样才能使用 SageMaker 批转换功能测试拟合后的模型。

In [ ]:
import pandas as pd

val_X = pd.DataFrame(train_X[:10000])
train_X = pd.DataFrame(train_X[10000:])

val_y = pd.DataFrame(train_y[:10000])
train_y = pd.DataFrame(train_y[10000:])

test_y = pd.DataFrame(test_y)
test_X = pd.DataFrame(test_X)

SageMaker 中的 XGBoost 算法的参考文档要求训练集和验证集不包含标题或索引，并且每个样本的标签在前面。

要详细了解此算法以及其他算法，请参阅 [Amazon SageMaker 开发人员文档](https://docs.aws.amazon.com/sagemaker/latest/dg/)。

In [ ]:
# First we make sure that the local directory in which we'd like to store the training and validation csv files exists.
data_dir = '../data/xgboost'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
# First, save the test data to test.csv in the data_dir directory. Note that we do not save the associated ground truth
# labels, instead we will use them later to compare with our model output.

pd.concat([test_y, test_X], axis=1).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)
pd.concat([val_y, val_X], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([train_y, train_X], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [ ]:
# To save a bit of memory we can set text_X, train_X, val_X, train_y and val_y to None.

train_X = val_X = train_y = val_y = None

### (TODO) 将训练/验证文件上传到 S3

Amazon S3 服务允许我们存储文件，内置训练模型（例如我们将使用的 XGBoost 模型）和自定义模型（例如我们稍后将查看的模型）可以访问这些文件。

对于此任务以及将使用 SageMaker 完成的大多数其他任务，我们可以使用两种方法。一种是使用 SageMaker 的低阶方法，低阶方法要求我们知道在 SageMaker 环境中出现的每个对象。第二种是使用高阶方法，SageMaker 会代替我们做出一些选择。低阶方法的好处是给用户带来了很高的灵活性，而高阶方法使开发速度快多了。对我们来说，我们将使用高阶方法，但是也可以使用低阶方法。

方法 `upload_data()` 是代表当前 SageMaker 会话的对象的成员。该方法会将数据上传到默认存储桶（如果不存在的话，将会创建），并放入由 key_prefix 变量指定的路径下。上传数据文件后，你可以转到 S3 控制台并看看文件上传到哪了。

要查看其他资源，请参阅 [SageMaker API 文档](http://sagemaker.readthedocs.io/en/latest/)以及 [SageMaker 开发人员指南](https://docs.aws.amazon.com/sagemaker/latest/dg/)。

In [ ]:
import sagemaker

session = sagemaker.Session() # Store the current SageMaker session

# S3 prefix (which folder will we use)
prefix = 'sentiment-xgboost'

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

### (TODO) 创建优化超参数的 XGBoost 模型

上传数据后，下面开始创建 XGBoost 模型。与 Boston Housing notebook 一样，第一步是创建一个 estimator 对象，它将用作超参数优化作业的基础模型。

In [ ]:
from sagemaker import get_execution_role

# Our current execution role is require when creating the model as the training
# and inference code will need to access the model artifacts.
role = get_execution_role()

In [ ]:
# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(session.boto_region_name, 'xgboost')

In [ ]:
# TODO: Create a SageMaker estimator using the container location determined in the previous cell.
#       It is recommended that you use a single training instance of type ml.m4.xlarge. It is also
#       recommended that you use 's3://{}/{}/output'.format(session.default_bucket(), prefix) as the
#       output path.

xgb = None

# TODO: Set the XGBoost hyperparameters in the xgb object. Don't forget that in this case we have a binary
#       label so we should be using the 'binary:logistic' objective.



### (TODO) 创建超参数优化器

设置好基础评估器后，我们需要构建超参数优化器对象，然后要求 SageMaker 使用该对象构建一个超参数优化作业。

**注意：**训练一个情感分析 XGBoost 模型的时间比训练 Boston Housing XGBoost 模型的时间要长，所以如果你不希望超参数优化作业的运行时间太长，请不要将模型（作业）总数设得太高。

In [ ]:
# First, make sure to import the relevant objects used to construct the tuner
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

# TODO: Create the hyperparameter tuner object

xgb_hyperparameter_tuner = None



### 拟合超参数优化器

构建了超参数优化器对象后，下面拟合各种模型并找到性能最佳的模型。

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

In [ ]:
xgb_hyperparameter_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

注意，优化作业已构建并在后台运行，如果你想查看训练作业的进度，请调用 `wait()` 方法。

In [ ]:
xgb_hyperparameter_tuner.wait()

### (TODO) 测试模型

运行超参数优化作业后，下面看看最佳模型的实际效果如何。我们将使用 SageMaker 的批转换功能。通过批转换功能可以轻松地对大型数据集进行推理，因为它并非实时执行。我们并不需要立即使用模型的结果，可以对大量样本进行推理。示例行业应用包括月末报告。这种推理方法的另一个用处是可以对整个测试集进行推理。

为了创建执行批转换作业的 transformer 对象，我们需要一个训练过的 estimator 对象。我们可以使用 `attach()` 方法将创建的 estimator 对象连接到最佳训练作业上。

In [ ]:
# TODO: Create a new estimator object attached to the best training job found during hyperparameter tuning

xgb_attached = None


我们已经将 estimator 对象连接到了正确的训练作业上，下面像之前一样创建 transformer 对象。

In [ ]:
# TODO: Create a transformer object from the attached estimator. Using an instance count of 1 and an instance type of ml.m4.xlarge
#       should be more than enough.

xgb_transformer = None


接下来执行转换作业。我们需要指定要发送的数据的类型，使 SageMaker 能够在后台正确地序列化数据。我们将向模型提供 csv 数据，所以指定为 `text/csv`。此外，如果我们提供的测试数据太大，无法一次性处理完，我们需要指定文件的拆分方式。因为数据集中的每行就是一个条目，所以我们将按照每行拆分输入数据。

In [ ]:
# TODO: Start the transform job. Make sure to specify the content type and the split type of the test data.


目前转换作业已经在运行，不过是在后台运行。因为我们要等待转换作业运行完毕，所以可以使用 `wait()` 方法查看运行进度。

In [ ]:
xgb_transformer.wait()

现在转换作业已经执行并且结果（每条影评的预测情感）已经保存到 S3 上。因为我们要在本地分析文件，所以通过一个 notebook 功能将文件复制到 `data_dir`。

In [ ]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

最后一步是读入模型的输出，将输出转换成可用的格式，我们希望情感为 `1`（正面）或 `0`（负面），然后与真实标签进行比较。

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

## 可选步骤：清理数据

SageMaker 上的默认 notebook 实例没有太多的可用磁盘空间。当你继续完成和执行 notebook 时，最终会耗尽磁盘空间，导致难以诊断的错误。完全使用完 notebook 后，建议删除创建的文件。你可以从终端或 notebook hub 删除文件。以下单元格中包含了从 notebook 内清理文件的命令。

In [ ]:
# First we will remove all of the files contained in the data_dir directory
!rm $data_dir/*

# And then we delete the directory itself
!rmdir $data_dir

# Similarly we will remove the files in the cache_dir directory and the directory itself
!rm $cache_dir/*
!rmdir $cache_dir